# Aquatic-based atmopsheric correction for Sentinel-2 MSI<br>and Landsat-8/9 OlI/OLI2
### Top-of-Atmosphere to Remote-Sensing Reﬂectance

Created by: Michael Brechbühler (michael.brechbuehler@eawag.ch)<br>
Adapted from Geo-AquaWatch GEE project (https://github.com/gee-community/geo-aquawatch-water-quality)   

This script accesses the Google Earth Engine [Sentinel-2 MSI L1C (R_toa)](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED) and [Landsat-8/9 OLI/OLI2 Raw (Rad_atsensor)](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1) collections to Remote-sensing reflectane (R_rs). The atmospheric correction is based on the MAIN algorithm (see reference).

**Page, B.P., Olmanson, L.G. and Mishra, D.R., 2019.**<br>
    ***A harmonized image processing workflow using Sentinel-2/MSI and<br>
    Landsat-8/OLI for mapping water clarity in optically variable lake systems.<br>
    Remote Sensing of Environment, 231, p.111284.***

For Jupyter Notebook versions < 5.2.2, notebook has to be run with high rate limit:
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [1]:
# Import Packages and Authenticate
import ee

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhUjTvrA7fPVGhELGR9KpevbQLrRJ2q7FM5vwmR7YCTDcjhRDwt1IL0

Successfully saved authorization token.


### 1a) User inputs

In [2]:
# Target specific time frame
start_date, end_date = '2019-06-01', '2019-09-30'

## Cloud filtering
cloud_perc = 20 # Scene-based cloud thresold (%) based on metadata
cloud_perc_s2cloudless = 20 # Pixel-based cloud probability threshold (%) based on s2cloudless

### 2a) Initialize collections

In [5]:
# Import Collections Sentinel-2
msi = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

# Filter data
ic_msi = (
    msi
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc)
        .filter(ee.Filter.date(start_date, end_date))
        .sort('system:time_start')
)

ic_oli1 = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1")
        .filterMetadata('CLOUD_COVER', "less_than", cloud_perc)
        .filter(ee.Filter.date(start_date, end_date))
)

ic_oli2 = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1")
        .filterMetadata('CLOUD_COVER', "less_than", cloud_perc)
        .filter(ee.Filter.date(start_date, end_date))
)

ic_oli = ic_oli1.merge(ic_oli2)


### 2b) Define processing functions (atmospheric correction, cloud-masking, water-masking)

In [26]:
## Landsat-8/9 OLI/OLI2
def atmcorr_main_oli(img):
    """Apply MAIN atmospheric correction to Landsat OLI-1/OLI-2 sensor."""
    # get platform
    platform = ee.String(img.get('SPACECRAFT_ID'))

    # define bands to be converted to radiance
    bands = ['B1','B2','B3','B4','B5','B6','B7']

    # tile footprint
    footprint = img.geometry()

    # DEM (either alos or srtm)
    dem_srtm = ee.Image('USGS/SRTMGL1_003').clip(footprint)
    dem_alos = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM').mosaic().clip(footprint)
    DEM = dem_srtm

    # ozone
    ozone = ee.ImageCollection('TOMS/MERGED')
    DU = ee.Image(ozone.filterDate(img.date().advance(-1, 'day'), img.date()).filterBounds(footprint).mean())

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'))
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1)
    JD = imgDate.difference(FOY,'day').int().add(1)

    # Define pi
    pi = ee.Image(3.141592)

    # Earth-Sun distance
    d = ee.Image.constant(img.get('EARTH_SUN_DISTANCE'))

    # Sun elevation
    SunEl = ee.Image.constant(img.get('SUN_ELEVATION'))

    # Sun azimuth
    SunAz = img.select('SAA').multiply(ee.Image(0.01))

    # Satellite zenith
    SatZe = img.select('VZA').multiply(ee.Image(0.01))
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos()
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin()

    # Satellite azimuth
    SatAz = img.select('VAA').multiply(ee.Image(0.01))
    
    # Sun zenith
    SunZe = img.select('SZA').multiply(ee.Image(0.01))
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image.constant(180))).cos() # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin() # in degrees

    # Relative azimuth
    RelAz = ee.Image(SunAz)
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos()

    # Pressure calculation
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25)

    ## Radiometric Calibration 

    # radiance_mult_bands
    rad_mult = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_MULT_BAND_1')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_2')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_3')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_4')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_5')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_6')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_7'))]
                        )).toArray(1)

    # radiance add band                         
    rad_add = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_ADD_BAND_1')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_2')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_3')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_4')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_5')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_6')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_7'))]
                        )).toArray(1)

    # create an empty image to save new radiance bands to
    imgArr = img.select(bands).toArray().toArray(1);
    Ltoa = imgArr.multiply(rad_mult).add(rad_add)

    # esun
    ESUN = (
        ee.Image.constant(197.24790954589844)
            .addBands(ee.Image.constant(201.98426818847656))
            .addBands(ee.Image.constant(186.12677001953125))
            .addBands(ee.Image.constant(156.95257568359375))
            .addBands(ee.Image.constant(96.04714965820312))
            .addBands(ee.Image.constant(23.8833221450863))
            .addBands(ee.Image.constant(8.04995873449635))
            .toArray().toArray(1)
    )

    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands])

    ## Ozone Correction 
    # Ozone coefficients
    koz = (
        ee.Image.constant(0.0039)
            .addBands(ee.Image.constant(0.0218))
            .addBands(ee.Image.constant(0.1078))
            .addBands(ee.Image.constant(0.0608))
            .addBands(ee.Image.constant(0.0019))
            .addBands(ee.Image.constant(0))
            .addBands(ee.Image.constant(0))
            .toArray().toArray(1)
    )

    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image.constant(1000));

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image.constant(1).divide(cosdSunZe)).add(ee.Image.constant(1).divide(cosdSatZe))).exp())

    # Rayleigh optical thickness
    bandCenter = (
        ee.Image(443).divide(1000)
            .addBands(ee.Image(483).divide(1000))
            .addBands(ee.Image(561).divide(1000))
            .addBands(ee.Image(655).divide(1000))
            .addBands(ee.Image(865).divide(1000))
            .addBands(ee.Image(1609).divide(1000))
            .addBands(ee.Number(2201).divide(1000))
            .toArray().toArray(1)
    )

     # create an empty image to save new Tr values to
    Tr = (
        (P.divide(Po))
            .multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4)))
            .multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))))
    )

    ## Fresnel Reflection 
    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001)
    sin_theta_j = sindSunZe.divide(ee.Image(1.333))

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi))

    theta_SZ = SunZe

    R_theta_SZ_s = (
        (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2))
            .divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)))
    )

    R_theta_V_s = ee.Image(0.0000000001)

    R_theta_SZ_p = (
        (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2))
            .divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)))
    )

    R_theta_V_p = ee.Image(0.0000000001)

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p))

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p))

    ## Rayleigh scattering phase function 
    # Sun-sensor geometry
    
    theta_neg = (
        ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe))
            .subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz))
    )

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi))

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz))

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi))

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos() # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos() # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))))

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))))

    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos));

    # Calulate Lr,
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe)
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom))

    # Rayleigh corrected radiance
    Lrc = (Lt.divide(ee.Image(10))).subtract(Lr)
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands])

    # Rayleigh corrected reflectance
    prc = Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe))
    prcImg = prc.arrayProject([0]).arrayFlatten([bands])
    rhorc = prc.arrayProject([0]).arrayFlatten([bands])

    # Aerosol Correction 
    # Bands in nm
    bands_nm = (
        ee.Image(443)
            .addBands(ee.Image(483))
            .addBands(ee.Image(561))
            .addBands(ee.Image(655))
            .addBands(ee.Image(865))
            .addBands(ee.Image(0))
            .addBands(ee.Image(0))
            .toArray().toArray(1)
    )

    # Lam in SWIR bands
    Lam_6 = LrcImg.select('B6')
    Lam_7 = LrcImg.select('B7')

    # Calculate aerosol type
    eps = (((((Lam_7).divide(ESUNImg.select('B7'))).log()).subtract(((Lam_6).divide(ESUNImg.select('B6'))).log())).divide(ee.Image(2201).subtract(ee.Image(1609))))

    # Calculate multiple scattering of aerosols for each band
    Lam = (
        Lam_7
        .multiply(((ESUN).divide(ESUNImg.select('B7'))))
        .multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2201)))).exp())
    )

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp()

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans)
    
    # water-leaving reflectance
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)))
    pwImg = pw.arrayProject([0]).arrayFlatten([bands])

    # Rrs
    Rrs = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,5))
    Rrs = Rrs.updateMask(Rrs.gt(0))

    return Rrs.set('system:time_start',img.get('system:time_start'))

def watermask_oli(img):
    """ Apply water-masking using CFMASK quality flag (QA_PIXEL)."""
    def bitwiseExtract(value, fromBit, toBit=None):
        if (toBit == None): toBit = fromBit
        maskSize = ee.Number(1).add(toBit).subtract(fromBit)
        mask = ee.Number(1).leftShift(maskSize).subtract(1)
        return(value.rightShift(fromBit).bitwiseAnd(mask))

    qa = img.select('QA_PIXEL')
    water = bitwiseExtract(qa, 7)
    return(img.updateMask(water))

def cloudmask_oli(img):
    """ Apply cloud-masking using CFMASK quality flag (QA_PIXEL)."""
    def bitwiseExtract(value, fromBit, toBit=None):
        if (toBit == None): toBit = fromBit
        maskSize = ee.Number(1).add(toBit).subtract(fromBit)
        mask = ee.Number(1).leftShift(maskSize).subtract(1)
        return(value.rightShift(fromBit).bitwiseAnd(mask))

    qa = img.select('QA_PIXEL')
    dilatedCloud =  bitwiseExtract(qa, 1)
    cirrus =        bitwiseExtract(qa, 2)
    cloud =         bitwiseExtract(qa, 3)
    cloudShadow =   bitwiseExtract(qa, 4)
    snow =          bitwiseExtract(qa, 5)
    clear =         bitwiseExtract(qa, 6)
    water =         bitwiseExtract(qa, 7)
    mask = (
        (cloud.Not())
            .And(dilatedCloud.Not())
            .And(cloudShadow.Not())
            #.And(water)
    )
    return(img.updateMask(mask))

## Sentinel-2 MSI
def atmcorr_main_msi(img):
    """Apply MAIN atmospheric correction to Sentinel-2 MSI sensor."""
    # get platform
    platform = ee.String(img.get('SPACECRAFT_NAME'))

    # msi bands
    bands = ['B1','B2','B3','B4','B5','B6','B7', 'B8', 'B8A', 'B11', 'B12']

    # rescle
    rescale = img.select(bands).divide(10000)

    # tile footprint
    footprint = rescale.geometry()

    # DEM 
    dem_srtm = ee.Image('USGS/SRTMGL1_003').clip(footprint)
    dem_alos = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').select('DSM').mosaic().clip(footprint)
    dem = dem_srtm

    # ozone
    ozone = ee.ImageCollection('TOMS/MERGED')
    DU = ee.Image(ozone.filterDate(img.date().advance(-1, 'day') ,img.date()).filterBounds(footprint).mean())

    # Julian Day
    imgDate = ee.Date(img.get('system:time_start'))
    FOY = ee.Date.fromYMD(imgDate.get('year'),1,1)
    JD = imgDate.difference(FOY,'day').int().add(1)

    # Define pi
    pi = ee.Image(3.141592)

    # Earth-Sun distance
    myCos = ((ee.Image(0.0172).multiply(ee.Image(JD).subtract(ee.Image(2)))).cos()).pow(2)
    cosd = myCos.multiply(pi.divide(ee.Image(180))).cos()
    d = ee.Image(1).subtract(ee.Image(0.01673)).multiply(cosd).clip(footprint)

    # sun azimuth
    SunAz = ee.Image.constant(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).clip(footprint)

    # sun zenith
    SunZe = ee.Image.constant(img.get('MEAN_SOLAR_ZENITH_ANGLE')).clip(footprint)
    cosdSunZe = SunZe.multiply(pi.divide(ee.Image(180))).cos() # in degrees
    sindSunZe = SunZe.multiply(pi.divide(ee.Image(180))).sin() # in degrees

    # sat zenith
    SatZe = ee.Image.constant(img.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')).clip(footprint)
    cosdSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).cos()
    sindSatZe = (SatZe).multiply(pi.divide(ee.Image(180))).sin()
  
    # sat azimuth
    SatAz = ee.Image.constant(img.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')).clip(footprint)

    # relative azimuth
    RelAz = SatAz.subtract(SunAz)
    cosdRelAz = RelAz.multiply(pi.divide(ee.Image(180))).cos()

    # Pressure
    P = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(dem)).pow(5.25588)).multiply(0.01)
    Po = ee.Image(1013.25)
    
    ESUN = ee.Image(ee.Array([ee.Image(img.get('SOLAR_IRRADIANCE_B1')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B2')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B3')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B4')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B5')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B6')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B7')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B8')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B8A')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B11')),
                  ee.Image(img.get('SOLAR_IRRADIANCE_B12'))]
                  )).toArray().toArray(1)

    ESUN = ESUN.multiply(ee.Image(1))
    ESUNImg = ESUN.arrayProject([0]).arrayFlatten([bands])
    
    # create empty array for the images
    imgArr = rescale.select(bands).toArray().toArray(1)
    
    # pTOA to Ltoa
    Ltoa = imgArr.multiply(ESUN).multiply(cosdSunZe).divide(pi.multiply(d.pow(2)))

    # band centers
    bandCenter_s2a = (
        ee.Image(444).divide(1000)
        .addBands(ee.Image(496).divide(1000))
        .addBands(ee.Image(560).divide(1000))
        .addBands(ee.Image(664).divide(1000))
        .addBands(ee.Image(704).divide(1000))
        .addBands(ee.Image(740).divide(1000))
        .addBands(ee.Image(782).divide(1000))
        .addBands(ee.Image(835).divide(1000))
        .addBands(ee.Image(865).divide(1000))
        .addBands(ee.Image(1613).divide(1000))
        .addBands(ee.Image(2202).divide(1000))
        .toArray().toArray(1)
    )
    
    bandCenter_s2b = (
        ee.Image(442).divide(1000)
        .addBands(ee.Image(492).divide(1000))
        .addBands(ee.Image(559).divide(1000))
        .addBands(ee.Image(665).divide(1000))
        .addBands(ee.Image(703).divide(1000))
        .addBands(ee.Image(739).divide(1000))
        .addBands(ee.Image(779).divide(1000))
        .addBands(ee.Image(833).divide(1000))
        .addBands(ee.Image(864).divide(1000))
        .addBands(ee.Image(1610).divide(1000))
        .addBands(ee.Image(2185).divide(1000))
        .toArray().toArray(1)
    )

    bandCenter = ee.Image(
        ee.Algorithms.If(platform.equals('Sentinel-2A'), bandCenter_s2a, bandCenter_s2b)
    )

    # ozone coefficients
    koz = (
        ee.Image(0.0040)
            .addBands(ee.Image(0.0244))
            .addBands(ee.Image(0.1052))
            .addBands(ee.Image(0.0516))
            .addBands(ee.Image(0.0208))
            .addBands(ee.Image(0.0112))
            .addBands(ee.Image(0.0079))
            .addBands(ee.Image(0.0021))
            .addBands(ee.Image(0.0019))
            .addBands(ee.Image(0))
            .addBands(ee.Image(0))
            .toArray().toArray(1)
    )
                        
    # Calculate ozone optical thickness
    Toz = koz.multiply(DU).divide(ee.Image(1000))

    # Calculate TOA radiance in the absense of ozone
    Lt = Ltoa.multiply(((Toz)).multiply((ee.Image(1).divide(cosdSunZe)).add(ee.Image(1).divide(cosdSatZe))).exp())

    # Rayleigh optical thickness
    Tr = (
        (P.divide(Po))
            .multiply(ee.Image(0.008569).multiply(bandCenter.pow(-4)))
            .multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter.pow(-4)))))
    )

    # Specular reflection (s- and p- polarization states)
    theta_V = ee.Image(0.0000000001)
    sin_theta_j = sindSunZe.divide(ee.Image(1.333))

    theta_j = sin_theta_j.asin().multiply(ee.Image(180).divide(pi))
    theta_SZ = SunZe

    R_theta_SZ_s = (
        (((theta_SZ.multiply(pi.divide(ee.Image(180)))).subtract(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2))
            .divide((((theta_SZ.multiply(pi.divide(ee.Image(180)))).add(theta_j.multiply(pi.divide(ee.Image(180))))).sin().pow(2)))
    )

    R_theta_V_s = ee.Image(0.0000000001)

    R_theta_SZ_p = (
        (((theta_SZ.multiply(pi.divide(180))).subtract(theta_j.multiply(pi.divide(180)))).tan().pow(2))
            .divide((((theta_SZ.multiply(pi.divide(180))).add(theta_j.multiply(pi.divide(180)))).tan().pow(2)))
    )

    R_theta_V_p = ee.Image(0.0000000001)

    R_theta_SZ = ee.Image(0.5).multiply(R_theta_SZ_s.add(R_theta_SZ_p))

    R_theta_V = ee.Image(0.5).multiply(R_theta_V_s.add(R_theta_V_p))
  
    # Sun-sensor geometry
    theta_neg = (
        ((cosdSunZe.multiply(ee.Image(-1))).multiply(cosdSatZe))
            .subtract((sindSunZe).multiply(sindSatZe).multiply(cosdRelAz))
    )

    theta_neg_inv = theta_neg.acos().multiply(ee.Image(180).divide(pi))

    theta_pos = (cosdSunZe.multiply(cosdSatZe)).subtract(sindSunZe.multiply(sindSatZe).multiply(cosdRelAz))

    theta_pos_inv = theta_pos.acos().multiply(ee.Image(180).divide(pi))

    cosd_tni = theta_neg_inv.multiply(pi.divide(180)).cos(); # in degrees

    cosd_tpi = theta_pos_inv.multiply(pi.divide(180)).cos(); # in degrees

    Pr_neg = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni.pow(2))))

    Pr_pos = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi.pow(2))))
  
    # Rayleigh scattering phase function
    Pr = Pr_neg.add((R_theta_SZ.add(R_theta_V)).multiply(Pr_pos))

    # rayleigh radiance contribution
    denom = ee.Image(4).multiply(pi).multiply(cosdSatZe)
    Lr = (ESUN.multiply(Tr)).multiply(Pr.divide(denom))

    # rayleigh corrected radiance
    Lrc = Lt.subtract(Lr)
    LrcImg = Lrc.arrayProject([0]).arrayFlatten([bands])
    prcImg = (Lrc.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)))
    prcImg = prcImg.arrayProject([0]).arrayFlatten([bands])

    ## Aerosol Correction
    # Bands in nm
    bands_nm_s2a = (
        ee.Image(444)
            .addBands(ee.Image(496))
            .addBands(ee.Image(560))
            .addBands(ee.Image(664))
            .addBands(ee.Image(703))
            .addBands(ee.Image(740))
            .addBands(ee.Image(782))
            .addBands(ee.Image(835))
            .addBands(ee.Image(865))
            .addBands(ee.Image(0))
            .addBands(ee.Image(0)).toArray().toArray(1)
    )

    bands_nm_s2b = (
        ee.Image(442)
            .addBands(ee.Image(492))
            .addBands(ee.Image(559))
            .addBands(ee.Image(665))
            .addBands(ee.Image(703))
            .addBands(ee.Image(739))
            .addBands(ee.Image(779))
            .addBands(ee.Image(833))
            .addBands(ee.Image(864))
            .addBands(ee.Image(0))
            .addBands(ee.Image(0)).toArray().toArray(1)
    )

    bands_nm = ee.Image(
        ee.Algorithms.If(platform.equals('Sentinel-2A'), bands_nm_s2a, bands_nm_s2b)
    )

    # Lam in SWIR bands
    Lam_10 = LrcImg.select('B11') # = 0
    Lam_11 = LrcImg.select('B12') # = 0 

    # Calculate aerosol type
    eps = ((((Lam_11).divide(ESUNImg.select('B12'))).log()).subtract(((Lam_10).divide(ESUNImg.select('B11'))).log())).divide(ee.Image(2190).subtract(ee.Image(1610)))

    # Calculate multiple scattering of aerosols for each band
    Lam = (Lam_11).multiply((ESUN).divide(ESUNImg.select('B12'))).multiply((eps.multiply(ee.Image(-1))).multiply((bands_nm.divide(ee.Image(2190)))).exp())

    # diffuse transmittance
    trans = Tr.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe)).exp();

    # Compute water-leaving radiance
    Lw = Lrc.subtract(Lam).divide(trans);

    # water-leaving reflectance 
    pw = (Lw.multiply(pi).multiply(d.pow(2)).divide(ESUN.multiply(cosdSunZe)));

    # remote sensing reflectance
    Rrs = (pw.divide(pi).arrayProject([0]).arrayFlatten([bands]).slice(0,9));

    # set negatives to null
    Rrs = Rrs.updateMask(Rrs.select('B1').gt(0))

    return Rrs.set('system:time_start', img.get('system:time_start'))

def cloudmask_msi(img):
    """ Apply cloud-masking using s2cloudless cloud probability."""
    # Get cloudmask from matching s2cloudless scene
    img_index = img.get('system:index')
    img_s2cloudless = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filter(ee.Filter.equals('system:index', img_index)).first()
    cloudmask = ee.Image(img_s2cloudless.get('cloud_mask')).select('probability').lt(cloud_perc_s2cloudless)
    # The masks for the 10m bands sometimes do not exclude bad data at
    # scene edges, so we apply masks from the 20m and 60m bands as well.
    mask_b8a = img.select('B8A').mask()
    mask_b9 = img.select('B9').mask()
    edgemask = mask_b8a.updateMask(mask_b9)
    return(img.updateMask(edgemask).updateMask(cloudmask))

def watermask_msi(img):
    """ Apply water-masking using NDVI."""
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    water = ndwi.gte(-0.15)
    return(img.updateMask(water))

In [29]:
## Process imagery
ic_msi_rrs = (
    ic_msi
        .map(cloudmask_msi)
        .map(watermask_msi)
        .map(atmcorr_main_msi)
        .sort('system:time_start')
)

ic_oli_rrs = (
    ic_oli
        .map(cloudmask_oli)
        .map(watermask_oli)
        .map(atmcorr_main_oli)
        .sort('system:time_start')
)

In [34]:
import geemap.foliumap as geemap
Map = geemap.Map()
Viz = {'min': 0, 'max': 0.015, 'palette': ['darkblue', 'blue', 'cyan', 'limegreen', 'yellow', 'orange', 'orangered', 'red', 'darkred']}

image = ic_oli_rrs.first()
Map.centerObject(image, 10)
Map.addLayer(image, Viz, 'OLI_Rrs')

EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [ ]:
Map